### Start Meshcat


In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


### Import URDF, Build both Robots, Define Frame


In [2]:
import os
import numpy as np
from pydrake.all import StartMeshcat
from rocky.build import build_robot_diagram
from rocky.ilqr import ILQRController, CostParams
from rocky.dynamics import ArmParams, forward_kinematics
from rocky.utils import *

urdf = "../rocky.urdf"

# build bundle from robot diagram
bundle = build_robot_diagram(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., 0.),
    meshcat=meshcat,
    disable_link_collisions=True,
)

# get plant, context, ally, enemy
plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

# create frames for everything
W = plant.world_frame()
enemy_target_frame = plant.GetFrameByName("enemy_target")
ally_target_frame  = plant.GetFrameByName("ally_target")
ally_wall_frame   = plant.GetFrameByName("wall", model_instance=ally)
ally_glove_frame  = plant.GetFrameByName("glove", model_instance=ally)
enemy_wall_frame  = plant.GetFrameByName("wall", model_instance=enemy)
enemy_elbow_frame = plant.GetFrameByName("elbow_marker", model_instance=enemy)
enemy_glove_frame = plant.GetFrameByName("glove", model_instance=enemy)

# get idx
ally_act_idxs = list(plant.GetJointActuatorIndices(ally))
enemy_act_idxs = list(plant.GetJointActuatorIndices(enemy))

# Initial position settings
enemy_config_mode = "custom"  # options: "random", "default", "custom"
custom_enemy_degs = [70.0, -160.0]

# Ally starts from a fixed configuration for repeatability
default_ally_degs = [45.0, -130.0]
ally_angles = np.deg2rad(default_ally_degs)
set_angles_for_instance(plant, ctx, ally, ally_angles)

# Enemy configuration selection
if enemy_config_mode == "random":
    enemy_angles = randomize_enemy_configuration()
elif enemy_config_mode == "default":
    enemy_angles = np.deg2rad([45.0, -140.0])
elif enemy_config_mode == "custom":
    enemy_angles = np.deg2rad(custom_enemy_degs)

set_angles_for_instance(plant, ctx, enemy, enemy_angles)

print("Enemy start angles (deg):", custom_enemy_degs)
print("Ally start angles (deg):", default_ally_degs)

# Advance simulation to 0 seconds
sim.AdvanceTo(0.0)

Enemy start angles (deg): [70.0, -160.0]
Ally start angles (deg): [45.0, -130.0]


## Boxing Simulation


### Set up controller parameters for ally and enemy

In [7]:
import os
import numpy as np
from pydrake.all import StartMeshcat
from rocky.build import build_robot_diagram
from rocky.ilqr import ILQRController, CostParams
from rocky.dynamics import ArmParams, forward_kinematics
from rocky.utils import *

urdf = "../rocky.urdf"

# build bundle from robot diagram
bundle = build_robot_diagram(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., 0.),
    meshcat=meshcat,
    disable_link_collisions=True,
)

# get plant, context, ally, enemy
plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

# create frames for everything
W = plant.world_frame()
enemy_target_frame = plant.GetFrameByName("enemy_target")
ally_target_frame  = plant.GetFrameByName("ally_target")
ally_wall_frame   = plant.GetFrameByName("wall", model_instance=ally)
ally_glove_frame  = plant.GetFrameByName("glove", model_instance=ally)
enemy_wall_frame  = plant.GetFrameByName("wall", model_instance=enemy)
enemy_elbow_frame = plant.GetFrameByName("elbow_marker", model_instance=enemy)
enemy_glove_frame = plant.GetFrameByName("glove", model_instance=enemy)

# get idx
ally_act_idxs = list(plant.GetJointActuatorIndices(ally))
enemy_act_idxs = list(plant.GetJointActuatorIndices(enemy))

# Initial position settings
enemy_config_mode = "custom"  # options: "random", "default", "custom"
custom_enemy_degs = [70.0, -160.0]

# Ally starts from a fixed configuration for repeatability
default_ally_degs = [45.0, -130.0]
ally_angles = np.deg2rad(default_ally_degs)
set_angles_for_instance(plant, ctx, ally, ally_angles)

# Enemy configuration selection
if enemy_config_mode == "random":
    enemy_angles = randomize_enemy_configuration()
elif enemy_config_mode == "default":
    enemy_angles = np.deg2rad([45.0, -140.0])
elif enemy_config_mode == "custom":
    enemy_angles = np.deg2rad(custom_enemy_degs)

set_angles_for_instance(plant, ctx, enemy, enemy_angles)

print("Enemy start angles (deg):", custom_enemy_degs)
print("Ally start angles (deg):", default_ally_degs)

# Advance simulation to 0 seconds
sim.AdvanceTo(0.0)


# Combined scenario: ally attacks first, then enemy attacks while ally defends
ally_attack_duration = 6.0
enemy_attack_duration = 5.0

# Default arm geometry parameters 
params = ArmParams()

# Parameteres
dt_mpc = 0.02

ally_attack_horizon = 50
ally_defense_horizon = 50

enemy_attack_horizon = 50
enemy_defense_horizon = 50

# When the enemy torque is limited, the ally is able to defend better
# When the enemy torque is the same, we need other ways for ally to defend
enemy_params = ArmParams(
    torque_limit=np.array([1.9, 1.9])
)

# At higher horizion, ally is able to defend better
# At lower horizion, ally is a bit behind
ally_attack_ctrl = ILQRController(
    horizon=ally_attack_horizon,
    dt=dt_mpc,
    params=params, 
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=425.0,
        r_min=0.01, 
        avoid_weight=-100.0,
        avoid_sigma=-0.08,
    ),
)

# Ally defense: use same controller type, but no avoidance (we WANT to block)
ally_defense_ctrl = ILQRController(
    horizon=ally_defense_horizon,
    dt=dt_mpc,
    params=params,
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=500.0,  # track blocking point strongly
        r_min=0.0,
        avoid_weight=-100.0,    # no obstacle avoidance term
        avoid_sigma=-0.5,
    ),
)

enemy_attack_ctrl = ILQRController(
    horizon=enemy_attack_horizon,
    dt=dt_mpc,
    params=enemy_params,
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=425.0,
        r_min=0.01,
        avoid_weight=10.0,
        avoid_sigma=0.08,
    ),
)

# Ally defense: use same controller type, but no avoidance (we WANT to block)
enemy_defense_ctrl = ILQRController(
    horizon=enemy_defense_horizon,
    dt=dt_mpc,
    params=params,
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=500.0,  # track blocking point strongly
        r_min=0.0,
        avoid_weight=0.0,    # no obstacle avoidance term
        avoid_sigma=0.0,
    ),
)

Enemy start angles (deg): [70.0, -160.0]
Ally start angles (deg): [45.0, -130.0]


### Simulation for Ally attacking while enemy defending, and then ally will block and parry enemy's attacks

In [68]:
# Reset recording
meshcat.DeleteRecording()
meshcat.StartRecording()

L1 = 0.6  # example, set to your values
L2 = 0.6  # example
r_max_block = 1.5 * (L1 + L2)
r_min_block = 0.05   # don't try to block closer than 15 cm from shoulder

lam_far = 0.5
lam_near = 0.8

################ Phase 1 ################
ally_u_prev = None
enemy_u_prev = None
enemy_def_u_prev = None

ally_mode = "attack"
ally_phase_end = sim.get_context().get_time() + ally_attack_duration

# Use a time step compatible with both ally attack and enemy defense
phase1_dt = min(ally_attack_ctrl.dt, enemy_defense_ctrl.dt)

print("Phase 1: Ally attacks, enemy defends")
while sim.get_context().get_time() < ally_phase_end:
    t = sim.get_context().get_time()    # current time

    # ---------------- Ally state and attack target (Ally frame) ----------------
    x_ally = get_ally_state(plant, ctx, ally)
    ally_ee_2d    = get_ally_pos_2d_from_model(x_ally, params)
    target_pos_2d = get_ally_target_pos_2d(plant, ctx, W, ally_wall_frame, enemy_target_frame)
    enemy_pos_2d  = get_enemy_pos_2d(plant, ctx, W, ally_wall_frame, enemy_glove_frame)

    # Check if ally already hit target
    hit = has_hit_target(ally_ee_2d, target_pos_2d)
    if hit:
        ally_mode = "hit"
        break

    # ---------------- Ally control (attack in Ally frame) ----------------
    x_pred, res = ally_attack_ctrl.step_mpc(
        x_ally,
        target_pos=target_pos_2d,
        enemy_pos=enemy_pos_2d,
        prev_u_trj=ally_u_prev,
    )
    if np.isfinite(res.cost) and res.converged:
        ally_u_prev = res.u_trj
        ally_u = res.u_trj[0]
    else:
        ally_u_prev = None
        ally_u = params.clip_u(-2.0 * x_ally[2:])

    # ---------------- Enemy defense (defending its own target in Enemy frame) ----------------
    # Enemy state
    x_enemy = get_enemy_state(plant, ctx, enemy)

    # Ally EE in Enemy frame (attacker)
    ally_pos_for_enemy = get_ally_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_glove_frame
    )

    # Enemy's own target (above its shoulder) in Enemy frame
    enemy_own_target_enemy = get_enemy_target_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, enemy_target_frame
    )

    # Compute blocking point between Ally EE and Enemy's own target (Enemy frame)
    dist_ET_enemy = np.linalg.norm(enemy_own_target_enemy - ally_pos_for_enemy)
    lam_enemy = lam_near + (lam_far - lam_near) * np.tanh(2.0 * (dist_ET_enemy - 0.2))

    raw_block_enemy = ally_pos_for_enemy + lam_enemy * (enemy_own_target_enemy - ally_pos_for_enemy)
    block_pos_enemy = clamp_radius(raw_block_enemy, r_min_block, r_max_block)

    # Enemy defense MPC (in Enemy frame)
    x_pred_def, res_def = enemy_defense_ctrl.step_mpc(
        x_enemy,
        target_pos=block_pos_enemy,    # block point in Enemy frame
        enemy_pos=np.zeros(2),         # no extra avoidance term
        prev_u_trj=enemy_def_u_prev,
    )

    if np.isfinite(res_def.cost) and res_def.converged:
        enemy_def_u_prev = res_def.u_trj
        u_enemy = res_def.u_trj[0]
    else:
        enemy_def_u_prev = None
        u_enemy = params.clip_u(-2.0 * x_enemy[2:])

    # ---------------- Apply torques and step simulation ----------------
    apply_combined_torques(plant, ctx, ally_u, u_enemy, ally_act_idxs, enemy_act_idxs)
    sim.AdvanceTo(t + phase1_dt)
################ Phase 2 ################
print("Phase 2: Enemy attacks, ally defends")

enemy_u_prev = None
ally_def_u_prev = None
enemy_mode = "attack"
phase2_end = sim.get_context().get_time() + enemy_attack_duration

defense_dt = min(enemy_attack_ctrl.dt, ally_defense_ctrl.dt)

# --- Continuous Parry FSM ---
ally_def_mode      = "block"   # "block" -> "parry" -> "recover" -> "block" ...
parry_start_time   = None
parry_duration     = 0.25
recover_start_time = None
recover_duration   = 0.25
block_tol          = 0.06      # EE must be within this distance of midpoint to trigger

while sim.get_context().get_time() < phase2_end:
    t = sim.get_context().get_time()

    # ---------------- Enemy state and target (Enemy frame) ----------------
    x_enemy = get_enemy_state(plant, ctx, enemy)
    enemy_target_pos_enemy = get_enemy_target_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_target_frame
    )
    ally_pos_for_enemy = get_ally_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_glove_frame
    )

    # ---------------- Ally state and enemy EE (Ally frame) ----------------
    x_ally = get_ally_state(plant, ctx, ally)
    enemy_pos_for_ally = get_enemy_pos_2d(
        plant, ctx, W, ally_wall_frame, enemy_glove_frame)
    enemy_target_for_ally = get_enemy_target_pos_2d_for_ally(
        plant, ctx, W, ally_wall_frame, ally_target_frame)
    ally_ee_2d = get_ally_pos_2d_from_model(x_ally, params)

    if np.any(np.isnan(x_enemy)) or np.any(np.isnan(x_ally)):
        print("ERROR: NaN state detected; stopping phase 2")
        break

    # Check if enemy hit target
    enemy_ee_2d_enemy = get_enemy_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, enemy_glove_frame)
    enemy_hit = has_hit_target(enemy_ee_2d_enemy, enemy_target_pos_enemy)
    if enemy_hit:
        enemy_mode = "hit"

    # ---------------- Enemy attack control ----------------
    if enemy_mode == "attack":
        x_pred_e, res_e = enemy_attack_ctrl.step_mpc(
            x_enemy,
            target_pos=enemy_target_pos_enemy,
            enemy_pos=ally_pos_for_enemy,
            prev_u_trj=enemy_u_prev,
        )
        if np.isfinite(res_e.cost) and res_e.converged:
            enemy_u_prev = res_e.u_trj
            u_enemy = res_e.u_trj[0]
        else:
            enemy_u_prev = None
            u_enemy = params.clip_u(-2.0 * x_enemy[2:])
    else:
        u_enemy = np.zeros(2)

    # ---------------- Blocking midpoint ----------------
    dist_ET = np.linalg.norm(enemy_target_for_ally - enemy_pos_for_ally)
    lam = lam_near + (lam_far - lam_near) * np.tanh(2.0 * (dist_ET - 0.2))
    raw_block = enemy_pos_for_ally + lam * (enemy_target_for_ally - enemy_pos_for_ally)
    block_pos_2d = clamp_radius(raw_block, r_min_block, r_max_block)

    # ---------------- Continuous Parry FSM ----------------
    dist_block = np.linalg.norm(ally_ee_2d - block_pos_2d)

    if ally_def_mode == "block":
        # If Ally is centered → start parry
        if dist_block < block_tol:
            ally_def_mode = "parry"
            parry_start_time = t

    elif ally_def_mode == "parry":
        # After parry time → recover
        if (t - parry_start_time) > parry_duration:
            ally_def_mode = "recover"
            recover_start_time = t

    elif ally_def_mode == "recover":
        # After short recovery → back to block (can trigger again)
        if (t - recover_start_time) > recover_duration:
            ally_def_mode = "block"

    # ---------------- Select Ally target ----------------
    if ally_def_mode == "parry":
        target_for_ally_ctrl = enemy_pos_for_ally
    else:
        target_for_ally_ctrl = block_pos_2d

    # ---------------- Ally MPC step (defense + parry) ----------------
    x_pred_a, res_a = ally_defense_ctrl.step_mpc(
        x_ally,
        target_pos=target_for_ally_ctrl,
        enemy_pos=np.zeros(2),
        prev_u_trj=ally_def_u_prev,
    )

    if np.isfinite(res_a.cost) and res_a.converged:
        ally_def_u_prev = res_a.u_trj
        u_ally = res_a.u_trj[0]
    else:
        ally_def_u_prev = None
        u_ally = params.clip_u(-2.0 * x_ally[2:])

    # ---------------- Apply torques and step ----------------
    apply_combined_torques(
        plant, ctx, u_ally, u_enemy, ally_act_idxs, enemy_act_idxs
    )
    sim.AdvanceTo(t + defense_dt)

    if enemy_hit:
        break

# End cleanup
apply_combined_torques(
    plant, ctx,
    u_ally=np.zeros(2),
    u_enemy=np.zeros(2),
    ally_act_idxs=ally_act_idxs,
    enemy_act_idxs=enemy_act_idxs,
)
meshcat.StopRecording()
meshcat.PublishRecording()
print("Combined scenario complete.")

Phase 1: Ally attacks, enemy defends
Phase 2: Enemy attacks, ally defends
Combined scenario complete.


### Begin Simulation for Ally attacking while enemy defending, and then ally will defend only

In [ ]:
# Reset recording
meshcat.DeleteRecording()
meshcat.StartRecording()

# ------------------------------------------------------------------
# Shared blocking parameters (used in both phases)
# ------------------------------------------------------------------
L1 = 0.6  # example, set to your values
L2 = 0.6  # example
r_max_block = 1.5 * (L1 + L2)
r_min_block = 0.05   # don't try to block closer than 15 cm from shoulder

lam_far = 0.5
lam_near = 0.8

################ Phase 1 ################
ally_u_prev = None
enemy_u_prev = None
enemy_def_u_prev = None

ally_mode = "attack"
ally_phase_end = sim.get_context().get_time() + ally_attack_duration

# Use a time step compatible with both ally attack and enemy defense
phase1_dt = min(ally_attack_ctrl.dt, enemy_defense_ctrl.dt)

print("Phase 1: Ally attacks, enemy defends")
while sim.get_context().get_time() < ally_phase_end:
    t = sim.get_context().get_time()    # current time

    # ---------------- Ally state and attack target (Ally frame) ----------------
    x_ally = get_ally_state(plant, ctx, ally)
    ally_ee_2d    = get_ally_pos_2d_from_model(x_ally, params)
    target_pos_2d = get_ally_target_pos_2d(plant, ctx, W, ally_wall_frame, enemy_target_frame)
    enemy_pos_2d  = get_enemy_pos_2d(plant, ctx, W, ally_wall_frame, enemy_glove_frame)

    # Check if ally already hit target
    hit = has_hit_target(ally_ee_2d, target_pos_2d)
    if hit:
        ally_mode = "hit"
        break

    # ---------------- Ally control (attack in Ally frame) ----------------
    x_pred, res = ally_attack_ctrl.step_mpc(
        x_ally,
        target_pos=target_pos_2d,
        enemy_pos=enemy_pos_2d,
        prev_u_trj=ally_u_prev,
    )
    if np.isfinite(res.cost) and res.converged:
        ally_u_prev = res.u_trj
        ally_u = res.u_trj[0]
    else:
        ally_u_prev = None
        ally_u = params.clip_u(-2.0 * x_ally[2:])

    # ---------------- Enemy defense (defending its own target in Enemy frame) ----------------
    # Enemy state
    x_enemy = get_enemy_state(plant, ctx, enemy)

    # Ally EE in Enemy frame (attacker)
    ally_pos_for_enemy = get_ally_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_glove_frame
    )

    # Enemy's own target (above its shoulder) in Enemy frame
    enemy_own_target_enemy = get_enemy_target_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, enemy_target_frame
    )

    # Compute blocking point between Ally EE and Enemy's own target (Enemy frame)
    dist_ET_enemy = np.linalg.norm(enemy_own_target_enemy - ally_pos_for_enemy)
    lam_enemy = lam_near + (lam_far - lam_near) * np.tanh(2.0 * (dist_ET_enemy - 0.2))

    raw_block_enemy = ally_pos_for_enemy + lam_enemy * (enemy_own_target_enemy - ally_pos_for_enemy)
    block_pos_enemy = clamp_radius(raw_block_enemy, r_min_block, r_max_block)

    # Enemy defense MPC (in Enemy frame)
    x_pred_def, res_def = enemy_defense_ctrl.step_mpc(
        x_enemy,
        target_pos=block_pos_enemy,    # block point in Enemy frame
        enemy_pos=np.zeros(2),         # no extra avoidance term
        prev_u_trj=enemy_def_u_prev,
    )

    if np.isfinite(res_def.cost) and res_def.converged:
        enemy_def_u_prev = res_def.u_trj
        u_enemy = res_def.u_trj[0]
    else:
        enemy_def_u_prev = None
        u_enemy = params.clip_u(-2.0 * x_enemy[2:])

    # ---------------- Apply torques and step simulation ----------------
    apply_combined_torques(plant, ctx, ally_u, u_enemy, ally_act_idxs, enemy_act_idxs)
    sim.AdvanceTo(t + phase1_dt)


################ Phase 2 ################
print("Phase 2: Enemy attacks, ally defends")

enemy_u_prev = None
ally_def_u_prev = None
enemy_mode = "attack"
phase2_end = sim.get_context().get_time() + enemy_attack_duration

defense_dt = min(enemy_attack_ctrl.dt, ally_defense_ctrl.dt)

while sim.get_context().get_time() < phase2_end:
    t = sim.get_context().get_time()

    # ---------------- Enemy state and target (Enemy frame) ----------------
    x_enemy = get_enemy_state(plant, ctx, enemy)
    enemy_target_pos_enemy = get_enemy_target_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_target_frame
    )   # Enemy target in Enemy frame (above Ally shoulder)
    ally_pos_for_enemy = get_ally_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, ally_glove_frame
    )   # Ally EE in Enemy frame

    # ---------------- Ally state and enemy EE (Ally frame) ----------------
    x_ally = get_ally_state(plant, ctx, ally)
    enemy_pos_for_ally      = get_enemy_pos_2d(plant, ctx, W, ally_wall_frame, enemy_glove_frame)
    enemy_target_for_ally   = get_enemy_target_pos_2d_for_ally(
        plant, ctx, W, ally_wall_frame, ally_target_frame
    )
    ally_ee_2d              = get_ally_pos_2d_from_model(x_ally, params)

    if np.any(np.isnan(x_enemy)) or np.any(np.isnan(x_ally)):
        print("ERROR: NaN state detected; stopping phase 2")
        break

    # Check if enemy hit its target (in Enemy frame)
    enemy_ee_2d_enemy = get_enemy_pos_2d_for_enemy(
        plant, ctx, W, enemy_wall_frame, enemy_glove_frame
    )
    enemy_hit = has_hit_target(enemy_ee_2d_enemy, enemy_target_pos_enemy)
    if enemy_hit:
        enemy_mode = "hit"

    # ---------------- Enemy control (attack in Enemy frame) ----------------
    if enemy_mode == "attack":
        x_pred_e, res_e = enemy_attack_ctrl.step_mpc(
            x_enemy,
            target_pos=enemy_target_pos_enemy,    # Enemy target in Enemy frame
            enemy_pos=ally_pos_for_enemy,         # Ally EE in Enemy frame
            prev_u_trj=enemy_u_prev,
        )
        if np.isfinite(res_e.cost) and res_e.converged:
            enemy_u_prev = res_e.u_trj
            u_enemy = res_e.u_trj[0]
        else:
            enemy_u_prev = None
            u_enemy = params.clip_u(-2.0 * x_enemy[2:])
    else:
        u_enemy = np.zeros(2)

    # Enemy EE and target in Ally frame (for blocking)
    enemy_pos_for_ally    = get_enemy_pos_2d(plant, ctx, W, ally_wall_frame, enemy_glove_frame)
    enemy_target_for_ally = get_enemy_target_pos_2d_for_ally(
        plant, ctx, W, ally_wall_frame, ally_target_frame
    )

    # Adaptive lam: closer to target as enemy approaches
    dist_ET = np.linalg.norm(enemy_target_for_ally - enemy_pos_for_ally)
    lam = lam_near + (lam_far - lam_near) * np.tanh(2.0 * (dist_ET - 0.2))

    raw_block = enemy_pos_for_ally + lam * (enemy_target_for_ally - enemy_pos_for_ally)
    block_pos_2d = clamp_radius(raw_block, r_min_block, r_max_block)

    x_pred_a, res_a = ally_defense_ctrl.step_mpc(
        x_ally,
        target_pos=block_pos_2d,      # Ally block point in Ally frame
        enemy_pos=np.zeros(2),        # avoidance off; not used
        prev_u_trj=ally_def_u_prev,
    )

    if np.isfinite(res_a.cost) and res_a.converged:
        ally_def_u_prev = res_a.u_trj
        u_ally = res_a.u_trj[0]
    else:
        ally_def_u_prev = None
        u_ally = params.clip_u(-2.0 * x_ally[2:])

    apply_combined_torques(plant, ctx, u_ally, u_enemy, ally_act_idxs, enemy_act_idxs)
    sim.AdvanceTo(t + defense_dt)

    if enemy_hit:
        break

# Stop and publish recording
apply_combined_torques(
    plant, ctx,
    u_ally=np.zeros(2),
    u_enemy=np.zeros(2),
    ally_act_idxs=ally_act_idxs,
    enemy_act_idxs=enemy_act_idxs,
)
meshcat.StopRecording()
meshcat.PublishRecording()
print("Combined scenario complete.")


Phase 1: Ally attacks, enemy defends
Phase 2: Enemy attacks, ally defends
Combined scenario complete.


### Replay

In [ ]:
meshcat.PublishRecording()

# Collocation

In [5]:
# Ally-only DirectCollocation using build_collocation_plant
import numpy as np
from pydrake.planning import DirectCollocation
from pydrake.all import Solve
from rocky.build import build_collocation_plant
from rocky.dynamics import ArmParams

# Current ally state from live sim
ally_q = plant.GetPositions(ctx, bundle.ally)[:2]
ally_dq = plant.GetVelocities(ctx, bundle.ally)[:2]
x0 = np.hstack([ally_q, ally_dq])

# Goal in joint space via simple IK toward enemy_target
target_pos = plant.CalcRelativeTransform(ctx, plant.world_frame(), ally_target_frame).translation()
xt, zt = float(target_pos[0]), float(target_pos[2])
params = ArmParams()
L1, L2 = params.l1, params.l2
cos_q2 = np.clip((xt**2 + zt**2 - L1**2 - L2**2) / (2 * L1 * L2), -1.0, 1.0)
q2_goal = np.arccos(cos_q2)
k1 = L1 + L2 * np.cos(q2_goal)
k2 = L2 * np.sin(q2_goal)
q1_goal = np.arctan2(zt, xt) - np.arctan2(k2, k1)
x_goal = np.hstack([np.array([q1_goal, q2_goal]), np.zeros(2)])

# Geometry-free planning plant (keeps collocation simple)
plan_plant, plan_ctx, model = build_collocation_plant("../rocky.urdf")
plan_plant.SetPositions(plan_ctx, model, ally_q)
plan_plant.SetVelocities(plan_ctx, model, ally_dq)

num_time_samples = 25
minimum_time_step = 0.05

coll = DirectCollocation(
    system=plan_plant,
    context=plan_ctx,
    num_time_samples=num_time_samples,
    minimum_time_step=minimum_time_step,
    maximum_time_step=0.05,
    input_port_index=plant.get_actuation_input_port(model).get_index(),
)
coll.AddEqualTimeIntervalsConstraints()
# coll.AddBoundingBoxConstraint(x0, x0, coll.initial_state())

u_lim = 30.0
# for k in range(25 - 1):
#     coll.AddBoundingBoxConstraint(-u_lim * np.ones(plan_plant.num_actuators()), u_lim * np.ones(plan_plant.num_actuators()), coll.input(k))

Q = np.diag([30.0, 30.0, 3.0, 3.0])
R = 1e-2 * np.eye(plan_plant.num_actuators())
coll.AddFinalCost((coll.state() - x_goal).T @ Q @ (coll.state() - x_goal))
for k in range(num_time_samples - 1):
    coll.AddRunningCost((coll.state() - x_goal).T @ Q @ (coll.state() - x_goal))
    coll.AddRunningCost(coll.input(k).T @ R @ coll.input(k))

result = Solve(coll.prog())
if not result.is_success():
    print("Collocation failed; using zero torques.")
    u_seq = np.zeros((num_time_samples - 1, plan_plant.num_actuators()))
else:
    u_seq = np.vstack([result.GetSolution(coll.input(k)) for k in range(num_time_samples - 1)])
    print("Collocation success; applying plan.")

# Apply ally torques in the live simulator
# ally_act_idxs = list(plant.GetJointActuatorIndices(bundle.ally))
ally_act_idxs = [int(i) for i in plant.GetJointActuatorIndices(bundle.ally)]
dt = minimum_time_step
cur_t = sim.get_context().get_time()
for u_step in u_seq:
    tau = np.zeros(plant.num_actuators())
    tau[ally_act_idxs] = u_step
    plant.get_actuation_input_port().FixValue(ctx, tau)
    sim.AdvanceTo(cur_t + dt)
    cur_t = sim.get_context().get_time()
plant.get_actuation_input_port().FixValue(ctx, np.zeros(plant.num_actuators()))
print("Ally driven toward target with build_collocation_plant.")


Collocation success; applying plan.
Ally driven toward target with build_collocation_plant.
